<a href="https://colab.research.google.com/github/stefymojica/MACC/blob/main/Modelo_clinico.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Agente para la optimizacion de recursos en un modelo clinico
## Matematicas aplicadas y ciencias de la computacion
## Estudiante : Stefany Paola Mojica Melo

## Contextualizacion

- 1 tick corresponde 1 minutos, 20 ticks corresponde a 20 minutos y asi sucesivamente
- Hay 3 agentes:
  - pacientes
  - medicos
  - triaja
- Hay un modelo
  - Clinica donde los agentes van a interactuar
- Los pacientes llegan y el triaje asigna el nivel de riesgo dependiendo de la situacion del paciente
- A la clinica llegan cada 25 minutos un nuevo paciente
- Los doctores se demoran 15 minutos para descubrir que le sucede a un paciennte
- los pacientes llegan y el triaje les asigana un nivel de criticidad son 5 niveles.
  - nivel 1 - situacion de riesgo vital, requiere accion inmediata
  - nivel 2 - Emergencia. requiere atencion en un tiempo maximo de 30 minutos
  - nivel 3 - Urgencia. se debe atender en un tiempo no mayor a 30 minutos
  - nivel 4 - urgencia menor. Puede esperar hasta 120 minutos
  - nivel 5 - sin urgencia. puede esperar hasta 180 minutos

- En la clinica entre 1 y 2 pacientes llegan cada 25 minutos

## Instalar librerias

In [ ]:
%pip install mesa

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.6/239.6 kB 5.4 MB/s eta 0:00:00


In [ ]:
import mesa
print("Versión de Mesa instalada:", mesa.__version__)

Versión de Mesa instalada: 3.3.0


In [ ]:
from mesa import Agent, Model
from mesa.space import MultiGrid
from mesa.datacollection import DataCollector
import random
import heapq

## Creacion de Agentes y Modelo

In [58]:
class SchedulerManual:
    def __init__(self, model):
        self.model = model
        self.agents = []

    def add(self, agent):
        self.agents.append(agent)

    def step(self):
        random.shuffle(self.agents)
        for agent in self.agents:
            agent.step()

class Paciente(Agent):
    def __init__(self, unique_id, model):
        super().__init__(model)
        self.unique_id = unique_id
        self.prioridad = None
        self.atendido = False

    def step(self):
        # El paciente solo espera (o es atendido)
        pass

class Triaje(Agent):
    def __init__(self, unique_id, model):
        super().__init__(model)
        self.unique_id = unique_id

    def step(self):
        # Procesa exactamente un paciente por minuto si hay en la antesala
        if len(self.model.pacientes_sin_triaje) > 0:
            paciente = self.model.pacientes_sin_triaje.pop(0)
            self.asignar_prioridad(paciente)

    def asignar_prioridad(self, paciente):
        # Distribución ajustable
        r = random.random()
        if   r < 0.05: paciente.prioridad = 1
        elif r < 0.20: paciente.prioridad = 2
        elif r < 0.50: paciente.prioridad = 3
        elif r < 0.80: paciente.prioridad = 4
        else:          paciente.prioridad = 5

        # Encolar por prioridad (1 = más urgente)
        heapq.heappush(self.model.cola_prioridad_triaje, (paciente.prioridad, paciente))

class Doctor(Agent):
    def __init__(self, unique_id, model):
        super().__init__(model)
        self.unique_id = unique_id
        self.ocupado = False
        self.paciente_actual = None
        self.tiempo_restante = 0  # minutos

    def step(self):
        # Si está atendiendo, corre el reloj
        if self.ocupado:
            self.tiempo_restante -= 1
            if self.tiempo_restante <= 0:
                self.ocupado = False
                self.paciente_actual = None
            return

        # Si está libre, toma el más urgente (si hay)
        if len(self.model.cola_prioridad_triaje) > 0:
            _, paciente = heapq.heappop(self.model.cola_prioridad_triaje)
            self.atender(paciente)

    def atender(self, paciente):
        self.ocupado = True
        self.paciente_actual = paciente
        paciente.atendido = True
        self.model.pacientes_atendidos += 1

        # Minutos por prioridad (ticks = minutos)
        tiempo_por_prioridad = {1: 30, 2: 20, 3: 15, 4: 10, 5: 8}
        self.tiempo_restante = tiempo_por_prioridad[paciente.prioridad]

class Clinica(Model):
    def __init__(self, n_doctores, semilla=None):
        super().__init__()
        if semilla is not None:
            random.seed(semilla)

        self.n_doctores = n_doctores
        self.pacientes_sin_triaje = []   # antesala (sin prioridad)
        self.cola_prioridad_triaje = []  # heap de (prioridad, paciente)
        self.pacientes_atendidos = 0
        self.tick = 0  # minutos

        self.schedule = SchedulerManual(self)

        # Doctores
        for d in range(n_doctores):
            self.schedule.add(Doctor(f"D{d}", self))

        # Triaje
        self.triaje = Triaje("T1", self)
        self.schedule.add(self.triaje)

    def _llegada_ola(self):
        # Cada 25 minutos, llega una ola de 1–2 pacientes
        if self.tick % 25 == 0:
            nuevos = random.randint(1, 2)
            for i in range(nuevos):
                pid = f"P{self.tick}_{i}"
                p = Paciente(pid, self)
                self.schedule.add(p)
                self.pacientes_sin_triaje.append(p)

    def step(self):
        # 1 tick = 1 minuto
        self._llegada_ola()
        self.schedule.step()
        self.tick += 1

    def mostrar_estado(self):
        h = self.tick // 60
        m = self.tick % 60
        print(f"\n===== MINUTO {self.tick} ({h:02d}:{m:02d}) =====")

        # Cola (prioridad, id paciente)
        if len(self.cola_prioridad_triaje) == 0:
            print("Cola de espera: VACÍA")
        else:
            cola_legible = [(prio, pac.unique_id) for (prio, pac) in self.cola_prioridad_triaje]
            print("Cola de espera (prioridad, paciente):", cola_legible)

        # Doctores
        for ag in self.schedule.agents:
            if isinstance(ag, Doctor):
                if ag.ocupado:
                    print(f"👨‍⚕️ Doctor {ag.unique_id} atendiendo a {ag.paciente_actual.unique_id} "
                          f"(restan {ag.tiempo_restante} min)")
                else:
                    print(f"✅ Doctor {ag.unique_id} está libre")

        print(f"📈 Total atendidos: {self.pacientes_atendidos}")


In [59]:
if __name__ == "__main__":
    clinica = Clinica(n_doctores=2, semilla=42)
    for _ in range(120): # prueba con 2 horas de simulacion
        clinica.step()
        clinica.mostrar_estado()


===== MINUTO 1 (00:01) =====
Cola de espera: VACÍA
✅ Doctor D1 está libre
👨‍⚕️ Doctor D0 atendiendo a P0_0 (restan 15 min)
📈 Total atendidos: 1

===== MINUTO 2 (00:02) =====
Cola de espera: VACÍA
👨‍⚕️ Doctor D0 atendiendo a P0_0 (restan 14 min)
✅ Doctor D1 está libre
📈 Total atendidos: 1

===== MINUTO 3 (00:03) =====
Cola de espera: VACÍA
✅ Doctor D1 está libre
👨‍⚕️ Doctor D0 atendiendo a P0_0 (restan 13 min)
📈 Total atendidos: 1

===== MINUTO 4 (00:04) =====
Cola de espera: VACÍA
👨‍⚕️ Doctor D0 atendiendo a P0_0 (restan 12 min)
✅ Doctor D1 está libre
📈 Total atendidos: 1

===== MINUTO 5 (00:05) =====
Cola de espera: VACÍA
✅ Doctor D1 está libre
👨‍⚕️ Doctor D0 atendiendo a P0_0 (restan 11 min)
📈 Total atendidos: 1

===== MINUTO 6 (00:06) =====
Cola de espera: VACÍA
✅ Doctor D1 está libre
👨‍⚕️ Doctor D0 atendiendo a P0_0 (restan 10 min)
📈 Total atendidos: 1

===== MINUTO 7 (00:07) =====
Cola de espera: VACÍA
✅ Doctor D1 está libre
👨‍⚕️ Doctor D0 atendiendo a P0_0 (restan 9 min)
📈 Total